
# Email Search AI — Semantic Search, Vector DB & Retrieval-Augmented Generation (RAG)

**Goal:** Build a generative search system for an organization's email corpus to *find and validate past decisions, strategies, and data* across large email threads.

1. **Embedding Layer** — preprocessing, cleaning, chunking strategies, and embeddings (OpenAI or SentenceTransformers).
2. **Search Layer** — vector database (Chroma), query generation, caching, and reranking (cross-encoders).
3. **Generation Layer** — RAG prompt design, few-shot examples, and LLM response generation.


## 0) Requirements & Setup

Install packages (recommended in virtualenv or Colab):

```bash
pip install -U pip
pip install pandas numpy tqdm nltk regex nbformat jupyterlab
pip install chromadb langchain sentence-transformers transformers accelerate faiss-cpu
pip install openai  # if you plan to use OpenAI embeddings / LLMs
pip install cross-encoder  # for reranking (from sentence-transformers)
```


## 1) Pipeline Overview (high level)

**Embedding Layer**
- Parse emails, extract metadata (from, to, subject, date), and body.
- Clean (remove quoted text, signatures, long headers).
- Chunking strategies to experiment with:
  - Fixed-size token windows (e.g., 200–500 tokens) with overlap (e.g., 50 tokens).
  - Sentence-based chunking (group by N sentences).
  - Thread-based chunking (keep entire thread together).
  - Semantically-aware chunking (use paragraph breaks + sentence-transformers clustering).
- Embedding model choices:
  - OpenAI embeddings (e.g., `text-embedding-3-small` / `text-embedding-3-large`) — higher quality, API needed.
  - Local models: SentenceTransformers (`all-MiniLM-L6-v2`, `paraphrase-MPNet-...`) from Hugging Face.

**Search Layer**
- Index chunks in ChromaDB (or FAISS).
- Design 3 test queries that reflect real tasks (see examples below).
- Implement caching of query embeddings and top-k results (simple file or Redis-based cache).
- Re-ranking via a cross-encoder model (e.g., `cross-encoder/ms-marco-MiniLM-L-6-v2`).

**Generation Layer**
- Compose a RAG prompt that provides: task instruction, retrieved context (top chunks), citation markers, and few-shot examples.
- Use LLM (OpenAI Chat or local Llama-like LLM) to produce final answer.
- Include an explanation of which chunks were used (traceability).


In [1]:
import os
import pandas as pd
from sentence_transformers import SentenceTransformer, CrossEncoder
import chromadb
from openai import OpenAI

In [6]:
def ingest_df(df, text_column="text", collection_name="emails"):
    embedder = SentenceTransformer('all-MiniLM-L6-v2')
    client = chromadb.PersistentClient(path="chroma_db")
    collection = client.get_or_create_collection(collection_name)

    # Clear existing data
    existing = collection.get()
    if "ids" in existing and existing["ids"]:
        collection.delete(ids=existing["ids"])
        print(f"Cleared {len(existing['ids'])} old items.")

    docs = df[text_column].tolist()
    embeddings = embedder.encode(docs).tolist()
    ids = [f"email_{i}" for i in range(len(docs))]

    collection.add(documents=docs, embeddings=embeddings, ids=ids)
    print(f"Ingested {len(docs)} records into collection '{collection_name}'.")

In [7]:
dataset_path = "D:/GenAI/Email_Search_AI/enron_emails.csv"

if os.path.exists(dataset_path):
    print(f"Found dataset at: {dataset_path}")
    df = pd.read_csv(dataset_path)
    text_col = next((c for c in df.columns if c.lower() in ["text", "body", "content", "message"]), None)
    if text_col:
        ingest_df(df, text_column=text_col, collection_name="emails")
    else:
        print(" No valid text column found.")
else:
    print("No dataset found. Using manual input mode.")

if 'email' in df.columns:
    sample_texts = df['email'].dropna().tolist()
elif 'body' in df.columns:
    sample_texts = df['body'].dropna().tolist()
else:
    raise ValueError("CSV must contain a column named 'email' or 'body'")

print(f"Loaded {len(sample_texts)} emails from CSV.")

Found dataset at: D:/GenAI/Email_Search_AI/enron_emails.csv
Cleared 1000 old items.
Ingested 1000 records into collection 'emails'.
Loaded 1000 emails from CSV.


In [8]:
embedder = SentenceTransformer('all-MiniLM-L6-v2')
client = chromadb.PersistentClient(path="chroma_db")
collection = client.get_or_create_collection("emails")


df = pd.DataFrame({"text": sample_texts})
ingest_df(df, text_column="text", collection_name="emails")

# Clear existing collection safely
try:
    all_data = collection.get()
    if 'ids' in all_data and all_data['ids']:
        collection.delete(ids=all_data['ids'])
        print(f"Cleared {len(all_data['ids'])} existing items.")
    else:
        print("Collection is already empty.")
except Exception as e:
    print("Could not clear collection:", e)

# Embed & add sample_texts
embeddings = embedder.encode(sample_texts).tolist()
collection.add(documents=sample_texts, embeddings=embeddings, ids=[f"email_{i}" for i in range(len(sample_texts))])
print(f"Stored {len(sample_texts)} sample emails in vector DB.")

Cleared 1000 old items.
Ingested 1000 records into collection 'emails'.
Cleared 1000 existing items.
Stored 1000 sample emails in vector DB.


In [ ]:
# ------------------ Querying ------------------

# Reranker model to use for re-ranking (define here to avoid NameError)
RERANK_MODEL = "cross-encoder/ms-marco-MiniLM-L-6-v2"

query = "kubernetes deployment issues"
print(f"\nQuery set to: {query}")

qvec = embedder.encode([query])[0]
results = collection.query(query_embeddings=[qvec.tolist()], n_results=3)
retrieved_docs = results.get('documents',[[]])[0]


if len(retrieved_docs) > 0:
    pairs = [(query, doc) for doc in retrieved_docs]
    try:
        reranker = CrossEncoder(RERANK_MODEL)
        scores = reranker.predict(pairs)
    except Exception as e:
        # If the reranker model cannot be loaded, skip reranking and assign a neutral score
        print("Reranker unavailable, skipping reranking:", e)
        scores = [0.0] * len(pairs)

    ranked = sorted(zip(retrieved_docs, scores), key=lambda x: x[1], reverse=True)
    print("\nTop 3 Results from the Search Layer:")
    for i, (doc, sc) in enumerate(ranked[:3], 1):
        print(f"{i}. ({sc:.3f}) {doc[:500]}\n---")
else:
    print("No documents retrieved.")

# Generation: optional (OpenAI)
try:
    from openai import OpenAI
    client = OpenAI(api_key='API_KEY_HERE')  # Replace with your actual API key or use env var
    context = "\n\n".join([doc for doc, _ in ranked[:3]]) if 'ranked' in locals() else ""
    prompt = f"You are an assistant that answers queries using only the provided email content.\n\nQuery: {query}\n\nContext:\n{context}\n\nAnswer based only on the context." 
    response = client.chat.completions.create(model='gpt-4o-mini', messages=[{'role':'user','content':prompt}])
    print("\nFinal Generated Answer:\n")
    print(response.choices[0].message.content)
except Exception as e:
    print("\nSkipping LLM generation — OpenAI API not configured or unavailable.")
    print(e)

print('\n Done — Email Search AI workflow completed.')



Query set to: kubernetes deployment issues

Top 3 Results from the Search Layer:
1. (-11.322) The project launch has been postponed due to client feedback. We are reviewing the vendor performance metrics for this quarter. The project launch has been postponed due to client feedback. Ensure all documents are updated before the deadline. Security policy updates will be rolled out next week.
---
2. (-11.343) Please find the attached report for this week’s progress. The project launch has been postponed due to client feedback. The board has approved the new budget for Q2. The project launch has been postponed due to client feedback. Security policy updates will be rolled out next week.
---
3. (-11.358) The project launch has been postponed due to client feedback. Let’s schedule a follow-up meeting to discuss next steps. We are reviewing the vendor performance metrics for this quarter. Please find the attached report for this week’s progress. Security policy updates will be rolled out next